In [1]:
# Checklist:
# AWS emr-5.29.0
# MASTER r5d.8xlarge 1x, no EBS
# CORE r5d.8xlarge 4x, no EBS
# Custom bootstrap action: s3://ydatazian/bootstrap.sh
# Allow ssh in master node security group

In [1]:
import tqdm.notebook as tqdm
import numpy as np
import scipy
import sklearn
import matplotlib.pyplot as plt

# SparkSession

https://spark.apache.org/docs/2.4.4/api/python/pyspark.html

https://spark.apache.org/docs/2.4.4/api/python/pyspark.sql.html

In [2]:
import findspark
findspark.init()

import spark_utils
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext("yarn", "My App", conf=spark_utils.get_spark_conf())
se = SparkSession(sc)
spark_utils.print_ui_links()

NameNode: http://ec2-18-215-34-68.compute-1.amazonaws.com:50070
YARN: http://ec2-18-215-34-68.compute-1.amazonaws.com:8088
Spark UI: http://ec2-18-215-34-68.compute-1.amazonaws.com:20888/proxy/application_1590934394702_0001


# Register all tables for sql queries

In [3]:
from IPython.display import display
tables = ["clicks_test", "clicks_train", 
          "documents_categories", "documents_entities", "documents_meta", "documents_topics", 
          "events", "page_views", "page_views_sample", "promoted_content"]
for name in tqdm.tqdm(tables):
    df = se.read.parquet("s3://ydatazian/{}.parquet".format(name))
    df.registerTempTable(name)
    print(name)
    display(df.limit(3).toPandas())

clicks_test


,display_id,ad_id
0,16874594,66758
1,16874594,150083
2,16874594,162754


clicks_train


,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1


documents_categories


...

,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1


page_views_sample


,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1


promoted_content


,ad_id,document_id,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7
2,3,7692,3,7


# Prepare dataset for VW

We will predict a *click* based on:
- ad_id
- document_id
- campaign_id
- advertiser_id

In [4]:
%%time
se.sql("""
select 
    clicks_train.clicked,
    clicks_train.display_id,
    clicks_train.ad_id,
    promoted_content.document_id,
    promoted_content.campaign_id,
    promoted_content.advertiser_id
from clicks_train join promoted_content on clicks_train.ad_id = promoted_content.ad_id
""").write.parquet("/train_features.parquet", mode='overwrite')

CPU times: user 5.28 ms, sys: 0 ns, total: 5.28 ms
Wall time: 47.9 s


In [5]:
se.read.parquet("/train_features.parquet").show(5)

+-------+----------+------+-----------+-----------+-------------+
|clicked|display_id| ad_id|document_id|campaign_id|advertiser_id|
+-------+----------+------+-----------+-----------+-------------+
|      0|         1| 42337|     938164|       5969|         1499|
|      0|         1|139684|    1085937|      17527|         2563|
|      1|         1|144739|    1337362|      18488|         2909|
|      0|         1|156824|     992370|       7283|         1919|
|      0|         1|279295|    1670176|      27524|         1820|
+-------+----------+------+-----------+-----------+-------------+
only showing top 5 rows



In [6]:
# Format: [Label] [Importance] [Base] [Tag]|Namespace Features |Namespace Features ... |Namespace Features
# https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format
def vw_row_mapper(row):
    clicked = None
    features = []
    for k, v in row.asDict().items():
        if k == 'clicked':
            clicked = '1' if v == '1' else '-1'
        else:
            features.append(k + "_" + v)
    tag = row.display_id + "_" + row.ad_id
    return "{} {}| {}".format(clicked, tag, " ".join(features))

r = se.read.parquet("/train_features.parquet").take(1)[0]
print(r)
print(vw_row_mapper(r))

Row(clicked='0', display_id='1', ad_id='42337', document_id='938164', campaign_id='5969', advertiser_id='1499')
-1 1_42337| display_id_1 ad_id_42337 document_id_938164 campaign_id_5969 advertiser_id_1499


In [7]:
%%time
! hdfs dfs -rm -r /train_features.txt
(
    se.read.parquet("/train_features.parquet")
    .rdd
    .map(vw_row_mapper)
    .saveAsTextFile("/train_features.txt")
)

rm: `/train_features.txt': No such file or directory
CPU times: user 24.1 ms, sys: 25.4 ms, total: 49.5 ms
Wall time: 2min 41s


In [8]:
# copy file to local master node
! rm /mnt/train.txt
! hdfs dfs -getmerge /train_features.txt /mnt/train.txt
# preview local file
! head -n 5 /mnt/train.txt

rm: cannot remove '/mnt/train.txt': No such file or directory
-1 1_42337| display_id_1 ad_id_42337 document_id_938164 campaign_id_5969 advertiser_id_1499
-1 1_139684| display_id_1 ad_id_139684 document_id_1085937 campaign_id_17527 advertiser_id_2563
1 1_144739| display_id_1 ad_id_144739 document_id_1337362 campaign_id_18488 advertiser_id_2909
-1 1_156824| display_id_1 ad_id_156824 document_id_992370 campaign_id_7283 advertiser_id_1919
-1 1_279295| display_id_1 ad_id_279295 document_id_1670176 campaign_id_27524 advertiser_id_1820


# Train VW
https://vowpalwabbit.org/tutorials/getting_started.html

https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments

In [9]:
! ./vw -d /mnt/train.txt -b 24 -c -k --ftrl --passes 1 -f model --holdout_off --loss_function logistic --random_seed 42 --progress 8000000

final_regressor = model
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = /mnt/train.txt.cache
Reading datafile = /mnt/train.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.460142 0.460142      8000000      8000000.0  -1.0000  -1.2983        6
0.456159 0.452175     16000000     16000000.0  -1.0000  -1.4733        6
0.453326 0.447661     24000000     24000000.0  -1.0000  -2.3174        6
0.452628 0.450533     32000000     32000000.0   1.0000  -0.5719        6
0.452887 0.453924     40000000     40000000.0  -1.0000  -1.2648        6
0.452010 0.447625     48000000     48000000.0  -1.0000  -2.6227        6
0.451328 0.447232     56000000     56000000.0  -1.0000  -3.4451        6
0.450649 0.445902     64000000     64000000.0  -1.0000  -1.

In [10]:
# make prediction with VW
! echo "? tag1| ad_id_144739 document_id_1337362 campaign_id_18488 advertiser_id_2909" > /mnt/test.txt
! echo "? tag2| ad_id_156824 document_id_992370 campaign_id_7283 advertiser_id_1919" >> /mnt/test.txt
! ./vw -d /mnt/test.txt -i model -t -k -p /mnt/predictions.txt --progress 1000000 --link=logistic
# predicted probabilities of "1" class
! cat /mnt/predictions.txt

only testing
predictions = /mnt/predictions.txt
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /mnt/test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features

finished run
number of examples = 2
weighted example sum = 2.000000
weighted label sum = 0.000000
average loss = 5.674418
total feature number = 10
0.319263 tag1
0.036173 tag2


# Homework 2: Baseline VW model

Train a baseline model using the following features:
- **clicked**
- geo_location features (country, state, dma)
- day_of_week (from timestamp, use *date.isoweekday()*)
- ad_id
- campaign_id
- advertiser_id
- ad_document_id
- display_document_id
- platform

Make submission to Kaggle to know your leaderboard score

If you want to create a dev set, make a 90%/10% split of training data by display_id

In [117]:
%%time
se.sql("""
with base_features as (
select 
    clicks_train.clicked,
    clicks_train.display_id,
    clicks_train.ad_id,
    promoted_content.document_id as ad_document_id,
    promoted_content.campaign_id,
    promoted_content.advertiser_id,
    events.timestamp,
    events.geo_location,
    events.platform,
    events.document_id as display_document_id
from clicks_train join promoted_content on clicks_train.ad_id = promoted_content.ad_id
                  join events on clicks_train.display_id = events.display_id)
                  select * from base_features
""").write.parquet("/train_features_extended.parquet", mode='overwrite')

KeyboardInterrupt: 

In [126]:
%%time
se.sql("""
with base_features as (
select 
    clicks_train.clicked,
    clicks_train.display_id,
    clicks_train.ad_id,
    promoted_content.document_id as ad_document_id,
    promoted_content.campaign_id,
    promoted_content.advertiser_id,
    events.timestamp,
    events.geo_location,
    events.platform,
    events.document_id as display_document_id
    from clicks_train join promoted_content on clicks_train.ad_id = promoted_content.ad_id
                  join events on clicks_train.display_id = events.display_id),
                  
    document_categories as 
    (select document_id, collect_list(category_id) as categories 
    from documents_categories group by document_id
    ),
    
    document_topics as 
    (select document_id, collect_list(topic_id) as topics
    from documents_topics group by document_id
    )
    
    select base_features.*, document_categories.categories cats, document_topics.topics topics from base_features 
    join document_categories on base_features.ad_document_id = document_categories.document_id
    join document_topics on base_features.ad_document_id = document_topics.document_id
                  
""").write.parquet("/train_features_extended.parquet", mode='overwrite')

CPU times: user 5.45 ms, sys: 336 µs, total: 5.79 ms
Wall time: 1min 10s


In [130]:
from datetime import datetime

INIT_TIMESTAMP = 1465876799

def vw_row_mapper_extended(row):
    clicked = None
    features = []
    for k, v in row.asDict().items():
        if k == 'cats':
            for cat in v:
                features.append("category" + "_" + cat),
        elif k == 'topics':
            for topic in v:
                features.append("topic" + "_" + topic)
        elif k == 'geo_location':
            if v:
                args = v.split(">")
                if len(args) == 3:
                    country, state, dma = args
                    features.append("state" + "_" + state)
                elif len(args) == 2:
                    country, dma = args
                    features.append("dma" + "_" + dma)
                elif len(args) == 1:
                    country = args[0]
                else:
                    raise RuntimeError("Unknown geolocation format:", v)

                features.append("country" + "_" + country)
            
        elif k == 'timestamp':
            v = int(v)
            v += INIT_TIMESTAMP
            v = datetime.utcfromtimestamp(v).isoweekday()
            features.append('weekday' + "_" + str(v))
        elif k == 'clicked':
            clicked = '1' if v == '1' else '-1'
        else:
            features.append(k + "_" + v)
    tag = row.display_id + "_" + row.ad_id
    return "{} {}| {}".format(clicked, tag, " ".join(features))

r = se.read.parquet("/train_features_extended.parquet").take(1)[0]
print(r)
print()
print(vw_row_mapper_extended(r))

Row(clicked='0', display_id='10876513', ad_id='59936', ad_document_id='1009080', campaign_id='7839', advertiser_id='2158', timestamp='730711821', geo_location='US>IL>602', platform='2', display_document_id='2429708', cats=['1609', '1512'], topics=['123', '285'])

-1 10876513_59936| display_id_10876513 ad_id_59936 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_3 state_IL country_US platform_2 display_document_id_2429708 category_1609 category_1512 topic_123 topic_285


In [132]:
%%time
! hdfs dfs -rm -r /train_features_extended.txt
(
    se.read.parquet("/train_features_extended.parquet")
    .rdd
    .map(vw_row_mapper_extended)
    .saveAsTextFile("/train_features_extended.txt")
)

Deleted /train_features_extended.txt
CPU times: user 49.7 ms, sys: 21.2 ms, total: 70.9 ms
Wall time: 1min 7s


In [87]:
# copy file to local master node
! rm /mnt/train_extended.txt
! hdfs dfs -getmerge /train_features_extended.txt /mnt/train_extended.txt
# preview local file
! head -n 5 /mnt/train_extended.txt

-1 10827410_59937| display_id_10827410 ad_id_59937 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_3 state_CA country_US platform_2 display_document_id_2479608 category_id_1609
-1 10827410_59937| display_id_10827410 ad_id_59937 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_3 state_CA country_US platform_2 display_document_id_2479608 category_id_1512
-1 12006881_59937| display_id_12006881 ad_id_59937 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_2 state_CA country_US platform_2 display_document_id_2490786 category_id_1609
-1 12006881_59937| display_id_12006881 ad_id_59937 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_2 state_CA country_US platform_2 display_document_id_2490786 category_id_1512
-1 12032505_59936| display_id_12032505 ad_id_59936 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_6 state_MA country_US platform_2 display_document_id_36610 category_id_1609


In [88]:
! ./vw -d /mnt/train_extended.txt -b 24 -c -k --ftrl --passes 1 -f model_extended --holdout_off --loss_function logistic --random_seed 42 --progress 8000000

final_regressor = model_extended
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = /mnt/train_extended.txt.cache
Reading datafile = /mnt/train_extended.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.448344 0.448344      8000000      8000000.0   1.0000  -0.8772       12
0.456448 0.464551     16000000     16000000.0  -1.0000  -1.2751       12
0.451263 0.440895     24000000     24000000.0  -1.0000  -1.5126       12
0.461087 0.490556     32000000     32000000.0  -1.0000  -2.5421       12
0.444787 0.379587     40000000     40000000.0  -1.0000  -2.4905       12
0.448269 0.465680     48000000     48000000.0  -1.0000  -2.3497       12
0.450470 0.463673     56000000     56000000.0  -1.0000  -0.8038       12
0.450127 0.447727     64000000  

In [89]:
%%time
se.sql("""
select 
    "0" as clicked,
    clicks_test.display_id,
    clicks_test.ad_id,
    promoted_content.document_id as ad_document_id,
    promoted_content.campaign_id,
    promoted_content.advertiser_id,
    events.timestamp,
    events.geo_location,
    events.platform,
    events.document_id as display_document_id,
    documents_categories.category_id
from clicks_test join promoted_content on clicks_test.ad_id = promoted_content.ad_id
                  join events on clicks_test.display_id = events.display_id
                  join documents_categories on documents_categories.document_id = promoted_content.document_id
""").write.parquet("/test_features_extended.parquet", mode='overwrite')

CPU times: user 4.63 ms, sys: 0 ns, total: 4.63 ms
Wall time: 50.6 s


# Submitting to Kaggle

Obtain Kaggle API token: https://github.com/Kaggle/kaggle-api#api-credentials

Making a submission: https://github.com/Kaggle/kaggle-api#submit-to-a-competition

In [11]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
! echo '{"username":"stanleybar","key":"e9e37e881fb09d147d9bfac3fb1276d5"}' > ~/.kaggle/kaggle.json
! cat ~/.kaggle/kaggle.json
! chmod 600 /home/hadoop/.kaggle/kaggle.json

{"username":"stanleybar","key":"e9e37e881fb09d147d9bfac3fb1276d5"}


In [12]:
! aws s3 cp s3://ydatazian/sample_submission.csv .

download: s3://ydatazian/sample_submission.csv to ./sample_submission.csv


In [13]:
# https://www.kaggle.com/c/outbrain-click-prediction/overview/evaluation
# For each display_id in the test set, you must predict a space-delimited list of ad_ids, 
# ordered by decreasing likelihood of being clicked.
! head -n 5 ./sample_submission.csv

display_id,ad_id
16874594,66758 150083 162754 170392 172888 180797
16874595,8846 30609 143982
16874596,11430 57197 132820 153260 173005 288385 289122 289915
16874597,137858 143981 155945 180965 182039 285834 305790 308836


In [15]:
%%time
se.sql("""
select 
    "0" as clicked,
    clicks_test.display_id,
    clicks_test.ad_id,
    promoted_content.document_id,
    promoted_content.campaign_id,
    promoted_content.advertiser_id
from clicks_test join promoted_content on clicks_test.ad_id = promoted_content.ad_id
""").write.parquet("/test_features.parquet", mode='overwrite')

CPU times: user 1.13 ms, sys: 2.32 ms, total: 3.45 ms
Wall time: 44.8 s


In [16]:
%%time
! hdfs dfs -rm -r /test_features.txt
(
    se.read.parquet("/test_features.parquet")
    .rdd
    .map(vw_row_mapper)
    .saveAsTextFile("/test_features.txt")
)

rm: `/test_features.txt': No such file or directory
CPU times: user 22.8 ms, sys: 14.4 ms, total: 37.1 ms
Wall time: 2min 31s


In [17]:
# copy file to local master node
! rm /mnt/test.txt
! hdfs dfs -getmerge /test_features.txt /mnt/test.txt
# preview local file
! head -n 5 /mnt/test.txt

-1 16874594_66758| display_id_16874594 ad_id_66758 document_id_1051283 campaign_id_8949 advertiser_id_555
-1 16874594_150083| display_id_16874594 ad_id_150083 document_id_1358132 campaign_id_19045 advertiser_id_1913
-1 16874594_162754| display_id_16874594 ad_id_162754 document_id_1292723 campaign_id_17770 advertiser_id_2391
-1 16874594_170392| display_id_16874594 ad_id_170392 document_id_1083829 campaign_id_20943 advertiser_id_1731
-1 16874594_172888| display_id_16874594 ad_id_172888 document_id_1433954 campaign_id_1384 advertiser_id_16


In [18]:
! ./vw -d /mnt/test.txt -i model -t -k -p /mnt/predictions.txt --progress 1000000 --link=logistic
# predicted probabilities of "1" class
! head -n 5 /mnt/predictions.txt

only testing
predictions = /mnt/predictions.txt
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /mnt/test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.882649 0.882649      1000000      1000000.0  -1.0000   0.0992        6
0.892969 0.903290      2000000      2000000.0  -1.0000   0.2038        6
0.909992 0.944038      3000000      3000000.0  -1.0000   0.1904        6
0.930778 0.993135      4000000      4000000.0  -1.0000   0.1299        6
0.940643 0.980106      5000000      5000000.0  -1.0000   0.2314        6
0.936147 0.913665      6000000      6000000.0  -1.0000   0.2526        6
0.931488 0.903531      7000000      7000000.0  -1.0000   0.1489        6
0.934737 0.957483      8000000      8000000.0  -1.0000   0.1199 

In [19]:
! wc -l /mnt/predictions.txt

32225162 /mnt/predictions.txt


In [20]:
from collections import defaultdict
scores_by_display_id = defaultdict(dict)
for line in tqdm.tqdm(open('/mnt/predictions.txt')):
    score, tag = line.strip().split(" ")
    score = float(score)
    display_id, ad_id = tag.split("_")
    scores_by_display_id[display_id][ad_id] = score

In [21]:
with open("submission.txt", "w") as f:
    f.write("display_id,ad_id\n")
    for k, vs in tqdm.tqdm_notebook(scores_by_display_id.items()):
        f.write("{},{}\n".format(
            k, 
            " ".join([v[0] for v in sorted(vs.items(), key=lambda x: -x[1])])
        ))

In [22]:
! kaggle competitions submit -f submission.txt outbrain-click-prediction -m "baseline"

100%|########################################| 260M/260M [00:04<00:00, 58.2MB/s]
Successfully submitted to Outbrain Click Prediction

## SUBMIT EXTENDED MODEL

In [90]:
%%time
! hdfs dfs -rm -r /test_features_extended.txt
(
    se.read.parquet("/test_features_extended.parquet")
    .rdd
    .map(vw_row_mapper_extended)
    .saveAsTextFile("/test_features_extended.txt")
)



Deleted /test_features_extended.txt
CPU times: user 28.3 ms, sys: 101 ms, total: 129 ms
Wall time: 43.8 s


In [91]:
# copy file to local master node
! rm /mnt/test_extended.txt
! hdfs dfs -getmerge /test_features_extended.txt /mnt/test_extended.txt
# preview local file
! head -n 5 /mnt/test_extended.txt

-1 17678294_59937| display_id_17678294 ad_id_59937 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_6 state_NY country_US platform_2 display_document_id_2093324 category_id_1609
-1 17678294_59937| display_id_17678294 ad_id_59937 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_6 state_NY country_US platform_2 display_document_id_2093324 category_id_1512
-1 18284158_59937| display_id_18284158 ad_id_59937 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_3 state_NY country_US platform_2 display_document_id_1026091 category_id_1609
-1 18284158_59937| display_id_18284158 ad_id_59937 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_3 state_NY country_US platform_2 display_document_id_1026091 category_id_1512
-1 19037404_59937| display_id_19037404 ad_id_59937 ad_document_id_1009080 campaign_id_7839 advertiser_id_2158 weekday_2 country_US platform_2 display_document_id_2574546 category_id_1609


In [92]:
! ./vw -d /mnt/test_extended.txt -i model_extended -t -k -p /mnt/predictions_extended.txt --progress 1000000 --link=logistic
# predicted probabilities of "1" class
! head -n 5 /mnt/predictions_extended.txt

only testing
predictions = /mnt/predictions_extended.txt
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /mnt/test_extended.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.650104 1.650104      1000000      1000000.0  -1.0000   0.4100       12
1.364729 1.079355      2000000      2000000.0  -1.0000   0.0762       12
1.159788 0.749906      3000000      3000000.0  -1.0000   0.0358       12
1.047206 0.709460      4000000      4000000.0  -1.0000   0.1855       12
1.093285 1.277598      5000000      5000000.0  -1.0000   0.2591       12
1.054614 0.861260      6000000      6000000.0  -1.0000   0.0949       12
1.222118 2.227146      7000000      7000000.0  -1.0000   0.3512       12
1.161776 0.739375      8000000      8000000.0 

In [93]:
! wc -l /mnt/predictions_extended.txt

63922585 /mnt/predictions_extended.txt


In [94]:
from collections import defaultdict
scores_by_display_id = defaultdict(dict)
for line in tqdm.tqdm(open('/mnt/predictions_extended.txt')):
    score, tag = line.strip().split(" ")
    score = float(score)
    display_id, ad_id = tag.split("_")
    scores_by_display_id[display_id][ad_id] = score

KeyboardInterrupt: 

In [ ]:
with open("submission_extended.txt", "w") as f:
    f.write("display_id,ad_id\n")
    for k, vs in tqdm.tqdm_notebook(scores_by_display_id.items()):
        f.write("{},{}\n".format(
            k, 
            " ".join([v[0] for v in sorted(vs.items(), key=lambda x: -x[1])])
        ))

In [ ]:
! kaggle competitions submit -f submission_extended.txt outbrain-click-prediction -m "extended"